### Imports and loading 

In [1]:
import asyncio
import dspy
from prompt_optimization.seed_prompts import ATOMIZER_PROMPT, PLANNER_PROMPT, AGGREGATOR_PROMPT, ATOMIZER_DEMOS, PLANNER_DEMOS
from dspy import GEPA

from prompt_optimization import (
    get_default_config,
    LMConfig,
    patch_romaconfig,
    load_aimo_datasets,
    ComponentJudge,
    MetricWithFeedback,
    create_optimizer,
)
from prompt_optimization.seed_prompts import (
    ATOMIZER_PROMPT,
    ATOMIZER_DEMOS,
    PLANNER_PROMPT,
    PLANNER_DEMOS,
    AGGREGATOR_PROMPT,
)
from roma_dspy.config import load_config
from roma_dspy.core.engine.solve import RecursiveSolver
from roma_dspy.core.modules.recursive_solver import RecursiveSolverModule
from roma_dspy.utils import AsyncParallelExecutor
from litellm.litellm_core_utils import logging_worker

def _run_logging_inline(async_coroutine):
    try:
        loop = asyncio.get_running_loop()
        loop.create_task(async_coroutine)
    except RuntimeError:
        asyncio.run(async_coroutine)

logging_worker.GLOBAL_LOGGING_WORKER.enqueue = _run_logging_inline
dspy.settings.provide_traceback = True  # optional but mirrors the old notebook
opt_cfg = load_config(profile="test")

2025-10-16 16:57:44.950 | DEBUG    | roma_dspy.config.manager:load_config:57 - Loading config: path=None, profile=test, overrides=None, env_prefix=ROMA_
2025-10-16 16:57:44.951 | DEBUG    | roma_dspy.config.manager:load_config:66 - Initialized empty base config (defaults applied in validation)
2025-10-16 16:57:44.958 | DEBUG    | roma_dspy.config.manager:_load_yaml:129 - Loaded and cached config from config/defaults/config.yaml
2025-10-16 16:57:44.959 | DEBUG    | roma_dspy.config.manager:load_config:81 - Merged default config from config/defaults/config.yaml
2025-10-16 16:57:44.961 | DEBUG    | roma_dspy.config.manager:_load_yaml:129 - Loaded and cached config from config/profiles/test.yaml
2025-10-16 16:57:44.964 | DEBUG    | roma_dspy.config.manager:load_config:87 - Applied profile: test
2025-10-16 16:57:44.972 | DEBUG    | roma_dspy.config.manager:load_config:103 - Resolved interpolations
/Users/salahalzubi/cursor_projects/ROMA-DSPy/src/roma_dspy/config/schemas/root.py:117: UserWar

### Config LLMS

In [2]:
opt_cfg.atomizer_lm = LMConfig("cerebras/qwen-3-235b-a22b-instruct-2507", temperature=0.35, max_tokens=128_000)
opt_cfg.planner_lm = LMConfig("cerebras/qwen-3-235b-a22b-instruct-2507", temperature=0.3, max_tokens=128_000)
opt_cfg.executor_lm = LMConfig("cerebras/gpt-oss-120b", temperature=0.6, max_tokens=128_000)
opt_cfg.aggregator_lm = LMConfig("cerebras/gpt-oss-120b", temperature=0.4, max_tokens=64_000)
opt_cfg.judge_lm = LMConfig("openrouter/anthropic/claude-sonnet-4.5", temperature=0.75, max_tokens=128_000, cache=True)
opt_cfg.reflection_lm = LMConfig("openrouter/anthropic/claude-sonnet-4.5", temperature=0.9, max_tokens=64_000)

In [3]:
# Batch the knobs you used to tweak in the notebook.
opt_cfg.train_size = 32
opt_cfg.val_size = 8
opt_cfg.test_size = 8
opt_cfg.dataset_seed = 42
opt_cfg.max_metric_calls = 225
opt_cfg.num_threads = 8
opt_cfg.max_parallel = 4
opt_cfg.concurrency = 4
opt_cfg.max_depth = 1
opt_cfg.enable_logging = True

In [4]:
#Add few-shot examples + prompts
opt_cfg.agents.atomizer.signature_instructions = ATOMIZER_PROMPT
opt_cfg.agents.planner.signature_instructions = PLANNER_PROMPT
opt_cfg.agents.aggregator.signature_instructions = AGGREGATOR_PROMPT

### Init solvers and what not

In [5]:
solver = RecursiveSolver(
    config=opt_cfg,
    max_depth=opt_cfg.max_depth,
    enable_logging=opt_cfg.enable_logging,
    enable_checkpoints=False,
)
solver_module = RecursiveSolverModule(solver=solver)

2025-10-16 16:57:48.024 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:112 - Using default signature for atomizer
2025-10-16 16:57:48.026 | INFO     | roma_dspy.core.factory.agent_factory:create_agent:88 - Created atomizer agent (task_type=default, signature=default)
2025-10-16 16:57:48.027 | DEBUG    | roma_dspy.core.registry.agent_registry:register_agent:180 - Registered atomizer instance #1 (task_type=default)
2025-10-16 16:57:48.027 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:112 - Using default signature for planner
2025-10-16 16:57:48.029 | INFO     | roma_dspy.core.factory.agent_factory:create_agent:88 - Created planner agent (task_type=default, signature=default)
2025-10-16 16:57:48.030 | DEBUG    | roma_dspy.core.registry.agent_registry:register_agent:180 - Registered planner instance #2 (task_type=default)
2025-10-16 16:57:48.030 | DEBUG    | roma_dspy.core.factory.agent_factory:_resolve_signature:112 - Using default signature fo

In [6]:
judge = ComponentJudge(lm_config=opt_cfg.judge_lm)  # keyword required after the refactor
metric = MetricWithFeedback(judge)

In [7]:
train_set, val_set, test_set = load_aimo_datasets(
    train_size=opt_cfg.train_size,
    val_size=opt_cfg.val_size,
    test_size=opt_cfg.test_size,
    seed=opt_cfg.dataset_seed,
)

### Perform an eval on the val set

In [ ]:
# executor = AsyncParallelExecutor(max_concurrency=4)

# results = await executor.execute_batch(solver_module, test_set)

In [ ]:
print(results)

### Prompt tuning stuff

In [8]:
optimizer = GEPA(
    metric=metric,
    # auto="light",
    component_selector="round_robin",
    max_metric_calls=12,
    add_format_failure_as_feedback=True,
    num_threads=12,
    track_stats=True,
    log_dir="logs/aime_test",
    # use_wandb=True,
    # wandb_init_kwargs={"project": "aime_test"},
    reflection_minibatch_size=8,
    reflection_lm=dspy.LM(model="openrouter/anthropic/claude-sonnet-4.5", temperature=.75, max_tokens=128000)
)

In [ ]:
optimized_program = optimizer.compile(
    solver_module,
    trainset=train_set,
    valset=val_set,
)

2025/10/16 16:57:57 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 12 metric calls of the program. This amounts to 0.30 full evals on the train+val set.
2025/10/16 16:57:57 INFO dspy.teleprompt.gepa.gepa: Using 8 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.
2025-10-16 16:57:57.656 | DEBUG    | roma_dspy.core.modules.recursive_solver:named_predictors:115 - RecursiveSolverModule.named_predictors exported 5 predictors: ['atomizer__default___predictor.predict', 'planner__default___predictor.predict', 'executor__default___predictor.predict', 'aggregator__default___predictor.predict', 'verifier__default___predictor.predict']
2025-10-16 16:57:57.656 | DEBUG    | roma_dspy.core.modules.recursive_solver:nam

  0%|          | 0/8 [00:00<?, ?it/s]

2025-10-16 16:59:52.746 | DEBUG    | roma_dspy.core.engine.solve:async_event_solve:461 - Starting async_event_solve for task: %s
2025-10-16 16:59:52.746 | DEBUG    | roma_dspy.core.engine.solve:async_event_solve:461 - Starting async_event_solve for task: %s
2025-10-16 16:59:52.746 | DEBUG    | roma_dspy.core.engine.solve:async_event_solve:461 - Starting async_event_solve for task: %s
2025-10-16 16:59:52.746 | DEBUG    | roma_dspy.core.engine.solve:async_event_solve:461 - Starting async_event_solve for task: %s
2025-10-16 16:59:52.747 | DEBUG    | roma_dspy.core.engine.solve:async_event_solve:461 - Starting async_event_solve for task: %s
2025-10-16 16:59:52.748 | INFO     | roma_dspy.core.storage.file_storage:__init__:109 - Initialized FileStorage for execution: fa0b3ba9-95db-4776-b2b9-e5e86e4886fa at .tmp/sentient/executions/fa0b3ba9-95db-4776-b2b9-e5e86e4886fa
2025-10-16 16:59:52.749 | INFO     | roma_dspy.core.storage.file_storage:__init__:109 - Initialized FileStorage for execution:

Average Metric: 1.00 / 1 (100.0%):  12%|█▎        | 1/8 [00:02<00:15,  2.22s/it]

2025-10-16 16:59:55.789 | INFO     | roma_dspy.resilience.decorators:async_wrapper:283 - ModuleRuntime._async_execute_module async completed | duration=1.75s
2025-10-16 16:59:55.790 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: 17e7ec66...
2025-10-16 16:59:55.790 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: 17e7ec66...
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-413' coro=<LoggingWorker._worker_loop() done, defined at /Users/salahalzubi/cursor_projects/ROMA-DSPy/.venv/lib/python3.12/site-packages/litellm/litellm_core_utils/logging_worker.py:57> exception=RuntimeError('<Queue at 0x142a79490 maxsize=50000> is bound to a different event loop')>
Traceback (most recent call last):
  File "/Users/salahalzubi/cursor_projects/ROMA-DSPy/.venv/lib/python3.12/site-packages/litellm/litellm_core_utils/logging_worker

...

Feedback: **Strengths:**
1. Correct classification: The atomizer accurately identified the problem as non-atomic
2. Appropriate node type assignment: PLAN is the correct choice for this multi-step optimization problem
3. Basic reasoning provided: The component gave a rationale for its decision

**Areas for Improvement:**

1. **More Specific Reasoning**: The reasoning should be more detailed and problem-specific. Instead of generic statements like "requires a multi-step approach," provide concrete analysis:
   - "This problem requires: (1) analyzing the constraint |x₁| + |x₂| + ... + |x₁₀₀| = 1, (2) applying the constraint x₁ + x₂ + ... + x₁₀₀ = 0, (3) determining optimal distribution of values, (4) maximizing the specific difference x₇₆ - x₁₆"

2. **Identify Key Sub-problems**: The reasoning should explicitly identify what types of subtasks would be needed:
   - Constraint analysis
   - Optimization setup
   - Algebraic manipulation
   - Final calculation and simplification

3. **Compl

2025-10-16 17:02:15.542 | INFO     | roma_dspy.resilience.decorators:async_wrapper:283 - ModuleRuntime._async_execute_module async completed | duration=1.12s
2025-10-16 17:02:15.543 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: cbb9f82c...
2025-10-16 17:02:15.544 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: cbb9f82c...
2025-10-16 17:02:15.545 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: 11a7d599...
2025-10-16 17:02:15.545 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: 11a7d599...
2025-10-16 17:02:15.547 | DEBUG    | roma_dspy.core.registry.agent_registry:get_agent:226 - Registry fallback: executor instance #3, requested=THINK, using default
2025-10-16 17:02:15.547 | DEBUG    | roma_dspy.core.engine

Feedback: **Critical Fix Required:**
1. **Resolve Goal Mismatch**: Investigate why the atomizer is receiving a different goal than what's being executed. The `goal` parameter shows the trigonometric problem while the context's `overall_objective` shows the triangle geometry problem. The system must ensure consistency between these fields.

**Improvements to Atomizer Logic:**
2. **More Specific Analysis**: Instead of generic reasoning ("requires multiple steps"), the atomizer should identify what makes a task atomic vs. non-atomic:
   - For atomic tasks: "This can be solved with direct computation/formula application"
   - For non-atomic tasks: "This requires [specific reasoning]: finding intermediate values X and Y, then combining them"

3. **Leverage Context Better**: When context contains an `overall_objective` that differs from the immediate `goal`, the atomizer should explicitly reconcile this:
   - Is the goal a subtask of the overall objective?
   - Is this a completely different

2025/10/16 17:05:21 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for atomizer__default___predictor.predict: You are a task atomizer that determines whether a given goal can be executed directly (atomic) or requires decomposition into subtasks (non-atomic).

## Input Format
You will receive:
1. **goal**: A specific task or problem to be evaluated
2. **context**: Contains fundamental_context with:
   - overall_objective: The root problem being solved
   - temporal: Current date/time information
   - recursion: Current depth and maximum depth allowed
   - available_tools: Tools that can be used for execution

## Output Format
Provide three fields:
1. **reasoning**: Detailed analysis of why the task is atomic or non-atomic
2. **is_atomic**: Boolean (True/False)
3. **node_type**: Either "EXECUTE" (for atomic tasks) or "PLAN" (for non-atomic tasks)

## Atomicity Criteria

### A task is ATOMIC (node_type: EXECUTE) if:
- It can be solved with a single mathematical technique o

2025-10-16 17:07:35.398 | INFO     | roma_dspy.resilience.decorators:async_wrapper:283 - ModuleRuntime._async_execute_module async completed | duration=1.52s
2025-10-16 17:07:35.399 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: 7eeed02b...
2025-10-16 17:07:35.399 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: 7eeed02b...
2025-10-16 17:07:35.401 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: 680875d4...
2025-10-16 17:07:35.401 | DEBUG    | roma_dspy.core.signatures.base_models.task_node:model_copy:64 - [TaskNode.model_copy] Preserving task_id: 680875d4...
2025-10-16 17:07:35.402 | DEBUG    | roma_dspy.core.registry.agent_registry:get_agent:226 - Registry fallback: executor instance #3, requested=THINK, using default
2025-10-16 17:07:35.403 | DEBUG    | roma_dspy.core.engine

In [10]:
optimized_program

2025-10-16 17:08:09.080 | DEBUG    | roma_dspy.core.modules.recursive_solver:named_predictors:115 - RecursiveSolverModule.named_predictors exported 5 predictors: ['atomizer__default___predictor.predict', 'planner__default___predictor.predict', 'executor__default___predictor.predict', 'aggregator__default___predictor.predict', 'verifier__default___predictor.predict']


atomizer__default___predictor.predict = Predict(StringSignature(goal, context -> reasoning, is_atomic, node_type
    instructions='Signature for task atomization.'
    goal = Field(annotation=str required=True description='Task to atomize' json_schema_extra={'__dspy_field_type': 'input', 'desc': 'Task to atomize', 'prefix': 'Goal:'})
    context = Field(annotation=Union[str, NoneType] required=False default=None description='Execution context (XML)' json_schema_extra={'__dspy_field_type': 'input', 'desc': 'Execution context (XML)', 'prefix': 'Context:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    is_atomic = Field(annotation=bool required=True description='True if task can be executed directly' json_schema_extra={'__dspy_field_type': 'output', 'desc': 'True if task can be executed directly', 'prefix': 'Is Atomic:'})
    node_type = Field(an